In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math
from datetime import timedelta

from Clust.clust.ML.regression_JS import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.ML.common.common import p3_training as p3
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.data import df_set_data
from Clust.clust.data import data_interface

from Clust.setting import influx_setting_KETI as ins
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)

from clust.preprocessing.dataPreprocessing import DataPreprocessing
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ML.tool import scaler as ml_scaler

ImportError: cannot import name 'p3_training' from 'Clust.clust.ML.common.common' (unknown location)

# Set Param

In [ ]:
feature_list = ["out_blacksphere_temp", "out_coai", "out_h2s", 'out_humi', 'out_nh3', 'out_pm10', 'out_temp', 'out_ultraviolet_rays']
target_col = "out_pm10"

get_y_data_transformParameter ={
    "future_step": 1,# 0~x
    "past_step":24,# 1~y
    "feature_col": feature_list,
    "target_col":target_col 
}   

param = {
    "x_features" : 8, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

bk_name = "air_outdoor_kweather"
ms_name = "OC3CL200010"

shuffle = False
model_file_path = "./ada_rnn_model/test_adarnn_model_by_origin_data"

# 22.01.01 ~ 22.01.11 데이터 없음
test_start_time = "2022-01-01 00:00:00"
test_end_time = "2022-04-29 23:59:00"

# get Data

## Data Read

In [ ]:
origin_data = db_client.get_data_by_time(test_start_time, test_end_time, bk_name, ms_name)

In [ ]:
origin_data

In [ ]:
origin_data.info()

In [ ]:
# Check Frequency
freq = db_client.get_freq(bk_name, ms_name)
print(freq)

In [ ]:
origin_data = origin_data[feature_list]

## Data Refine

In [ ]:
# fill in the missing time index
refine_param = {'removeDuplication': {'flag': True} , 'staticFrequency': {'flag': True, 'frequency': "1H"}}
data_refine = DataPreprocessing().get_refinedData(origin_data, refine_param)

In [ ]:
data_refine.info()

In [ ]:
data_refine.head()

In [ ]:
day_start = data_refine.index[0] + timedelta(days=1) - timedelta(hours=data_refine.index[0].hour, 
                                                                 minutes=data_refine.index[0].minute, 
                                                                 seconds=data_refine.index[0].second)

In [ ]:
data_refine = data_refine[day_start:]

In [ ]:
data_refine

## Data Nan -> Mean

In [ ]:
method = "mean"
max_limit = {}
parameter = {}

data_refine_nan = BasicImputation(data_refine, method, max_limit, parameter).simpleMethod()

In [ ]:
data_refine_nan.info()

## Data Scale

In [ ]:
# set parameter
scaler_root_dir ='./scaler/'

clean_train_data_param = 'NoClean'

data_name_x = "AdaRNN_data_x"
feature_list_x= list(data_refine_nan.columns)
x_scaler_file_path = './scaler/AdaRNN_data_x/NoClean/minmax/4457f465de3d0e17dd9f547db5b454b7/scaler.pkl'

split_ratio = 0.8
scaler_param='scale'
scale_method='minmax'

In [ ]:
scaler_root_path_x = os.path.join(scaler_root_dir, data_name_x, clean_train_data_param)
test_x, scaler_x = ml_scaler.get_scaled_test_data(data_refine_nan[feature_list_x], x_scaler_file_path, scaler_param)
#test_y, scaler_y = ml_scaler.get_scaled_test_data(data_y[target], y_scaler_file_path, scaler_param)# No Scale

In [ ]:
test_x.tail()

## get Data y

In [ ]:
LSTMD = LSTMData()
test_x_arr, test_y_arr = LSTMD.transform_Xy_arr(test_x, get_y_data_transformParameter, "NoClean")

In [ ]:
test_x_arr.shape, test_y_arr.shape

In [ ]:
test_y_arr = test_y_arr.reshape(test_y_arr.shape[0],)

# Data Loader

In [ ]:
adarnn = ada_rnn.ClustAdaRnn(param)

In [ ]:
test_loader = adarnn.create_testloader(test_x_arr, test_y_arr, shuffle)

# Testing

In [ ]:
## model load
adarnn.load_model(model_file_path)

In [ ]:
## test
test_loss, test_loss_l1, test_loss_r = adarnn.test(test_loader)

In [ ]:
test_loss, test_loss_l1, test_loss_r